In [ ]:
from owlready2 import *
onto = get_ontology("https://protege.stanford.edu/ontologies/pizza/pizza.owl").load()

from flask import Flask, url_for
app = Flask(__name__)

@app.route('/')
def ontology_page():
    html  = """<html>
                <head>
                <meta charset="UTF-8">
                <title>Template 3</title>
                <meta name="description" content="A description of your website">
                <meta name="keywords" content="keyword1, keyword2, keyword3">
                <link href="%s" rel="stylesheet" type="text/css"> 
                </head>
                <body>
                <div id="wrapper">
                
                  <div id="header"> 
                    <div class="top_banner">
                      <h1>Zioła</h1>
                      <p>Zioła Slogan</p>
                    </div>
                  </div>
                  
                  <div id="page_content">
  """ %url_for('static', filename='css/style.css')
    html += """<h2>'%s' ontology</h2>""" % onto.base_iri
    html += """<h3>Root classes</h3>"""
    for Class in Thing.subclasses():
        html += """<p><a href="%s">%s</a></p>""" %(url_for("class_page", iri = Class.iri), Class.name)
    html += """
    </body>
    </div>
    </div>
    </html>
    """
    return html

@app.route('/class/<path:iri>')
def class_page(iri):
    Class = IRIS[iri]

    html = """<html>
                    <head>
                <meta charset="UTF-8">
                <title>Template 3</title>
                <meta name="description" content="A description of your website">
                <meta name="keywords" content="keyword1, keyword2, keyword3">
                <link href="{}" rel="stylesheet" type="text/css"> 
                </head>
                <body>
                
                <div id="wrapper">
                
                  <div id="header"> 
                    <div class="top_banner">
                      <h1>Zioła</h1>
                      <p>Zioła Slogan</p>
                    </div>
                  </div>
                  
                  <div id="page_content">
    
    <h2>'{}' class</h2>""".format(url_for('static', filename='css/style.css'), Class.name)
    
    html += """<h3>Superclasses</h3>"""    
    for SuperClass in Class.is_a:
        if isinstance(SuperClass, ThingClass):
            html += """<p><a href="%s">%s</a></p>""" %(url_for("class_page", iri = SuperClass.iri), SuperClass.name)
        else:
            html += """<p>%s</p>""" % SuperClass

    html += """<h3>Equivalent classes</h3>"""
    for EquivClass in Class.equivalent_to:
        html += """<p>%s</p>""" % EquivClass

    html += """<h3>Subclasses</h3>"""
    for SubClass in Class.subclasses():
        html += """<p><a href="%s">%s</a></p>""" %(url_for("class_page", iri = SubClass.iri), SubClass.name)
        html += """<h3>Individuals</h3>"""
        for individual in Class.instances():
            html += """<p><a href="%s">%s</a></p>""" %(url_for("individual_page", iri = individual.iri), individual.name)
    
    
    html += """    </body>
    </div>
    </div>
    </html>"""
    return html

@app.route('/individual/<path:iri>')
def individual_page(iri):
    individual = IRIS[iri]

    html = """<html><body><h2>'%s' individual</h2>""" %individual.name
    
    html += """<h3>Classes</h3>"""
    for Class in individual.is_a:
        html += """<p><a href="%s">%s</a></p>""" %(url_for("class_page", iri = Class.iri), Class.name)
    
    html += """<h3>Relations</h3>"""
    if isinstance(individual, onto.Country):
        html += """<p>shape = %s</p>""" % individual.has_shape 
        html += """<p>grouping = %s</p>""" % individual.has_grouping 
        if individual.gram_positive == True:
            html += """<p>Gram +</p>"""
        elif individual.gram_positive == False:
            html += """<p>Gram -</p>"""

    html += """</body></html>"""
    return html

import werkzeug.serving
werkzeug.serving.run_simple("localhost", 5000, app)

INFO:werkzeug: * Running on http://localhost:5000/ (Press CTRL+C to quit)
INFO:werkzeug:127.0.0.1 - - [16/Mar/2021 08:57:41] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [16/Mar/2021 08:57:41] "GET /static/css/style.css HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [16/Mar/2021 08:57:41] "GET /static/images/header.jpg HTTP/1.1" 304 -
